## Import packages

In [1]:
import os
import tensorflow as tf
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
from tensorflow.keras.optimizers import Adam
import json
import pandas as pd
from tensorflow.keras.regularizers import l2

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import *
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, GRU, BatchNormalization
import optuna
from tensorflow.keras.preprocessing.sequence import pad_sequences
from  VietnameseTextNormalizer.ReleasePython3 import VietnameseTextNormalizer
import py_vncorenlp
import re
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_contour
from optuna.visualization import plot_param_importances
import optuna.visualization as ov
import matplotlib.pyplot as plt

2023-12-14 04:25:33.368778: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Init Vietnamese Text Normalizer version 1.9.5
	Bùi Tấn Quang - langmaninternet@gmail.com


## Define environment variables

In [2]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
annotate = py_vncorenlp.VnCoreNLP(annotators=["wseg", "pos", "ner", "parse"], save_dir='./')

2023-12-13 11:39:14 INFO  WordSegmenter:24 - Loading Word Segmentation model
2023-12-13 11:39:14 INFO  PosTagger:23 - Loading POS Tagging model
2023-12-13 11:39:17 INFO  NerRecognizer:34 - Loading NER model
2023-12-13 11:39:29 INFO  DependencyParser:32 - Loading Dependency Parsing model


In [2]:
ORIGINAL_DATA ="./original_data/"
PREPROCESSED_DATA = "./preprocessed_data/preprocessed_data.json"
MODEL_FOLDER = "./models"
WORD2VEC_MODEL = "word2vec_model_100dim_10min.save"
DOC2VEC_MODEL = "doc2vec_model_100dim_10min.save"
STOPWORD = "./vietnamese-stopwords-dash.txt"
with open(STOPWORD, 'r', encoding='utf-8') as stop_word_file:
    stop_words = stop_word_file.read().splitlines()

In [3]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [4]:
sep = os.sep
import warnings
warnings.filterwarnings('ignore')

## Preprocess

In [6]:
def preprocess_text(text):
    text= VietnameseTextNormalizer.Normalize(text)
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\b(www\.[^\s]+|(?!https?://)[^\s]+\.[a-z]{2,})\b', '', text)
    # Remove phone numbers
    text = re.sub(r'\b\d{10,}\b', '', text)

    # Remove emails
    text = re.sub(r'\S+@\S+', '', text)
    annotations = annotate.annotate_text(text)
    all_tokens = []
    for sentence_index, sentence_annotations in annotations.items():
        all_tokens.extend(sentence_annotations)
    word_list  = [
        token['wordForm'] for token in all_tokens
        if isinstance(token, dict) and 'wordForm' in token and isinstance(token['wordForm'], str)
        and 'posTag' in token and isinstance(token['posTag'], str) and token['posTag'] in ['N', 'V', 'A']
        and 'nerLabel' in token and token['nerLabel'] not in ['B-PER']
    ]
    clean_words = [word.strip(',').strip().lower() for word in word_list  if word not in stop_words]
    clean_words = [re.sub(r'([^\s\w]|)+', '', sentence) for sentence in clean_words if sentence!='']
    return clean_words


## Load data

In [10]:
# Đường dẫn đến file JSON
json_file_path = PREPROCESSED_DATA

# Load dữ liệu từ file JSON vào DataFrame
df = pd.read_json(json_file_path, encoding='utf-8', lines=True)

# Hiển thị DataFrame
print(df.head())

                                   Processed_Content  Category
0  [lễ, chủ_tịch, tỉnh, trao, khen, chủ_tịch, tỉn...  Giao duc
1  [đề_xuất, giáo_dục, kỳ_vọng, đột_phá, dư_luận,...  Giao duc
2  [gần, cư_dân, mạng, truyền, đoạn, clip, ghi, h...  Giao duc
3  [căn_bệnh, thành_tích, địa_phương, biến, thi, ...  Giao duc
4  [phân_biệt, dạy, tác_phẩm, chương_trình, ban, ...  Giao duc


In [11]:
labels = df['Category'].tolist()
# news là 1 list mà mỗi phần tử là 1 bài báo (1 list các từ đã được phân tách)
news = df['Processed_Content'].tolist()

In [12]:
print(news[0])
print(len(news[0]))
print(labels[0])
print(len(news))
print(len(labels))


['lễ', 'chủ_tịch', 'tỉnh', 'trao', 'khen', 'chủ_tịch', 'tỉnh', 'khen', 'bí_thư', 'tỉnh_uỷ', 'ghi_nhận', 'nỗ_lực', 'cố_gắng', 'học_tập', 'hoạt_động', 'xã_hội', 'đại_diện', 'lãnh_đạo', 'tỉnh', 'trao', 'thư', 'khen', 'khen', 'lãnh_đạo', 'tỉnh_uỷ', 'tỉnh', 'em', 'học_sinh', 'lớp', 'chuyên', 'trường', 'chuyên', 'sở_hữu', 'bảng_vàng', 'thành_tích', 'liên_tục', 'học_sinh', 'giỏi', 'giải', 'học_sinh', 'giỏi', 'quốc_gia', 'môn', 'tiếng', 'huy_chương', 'tiếng', 'quốc_gia', 'huy_chương', 'năng', 'tiếng', 'anh', 'quốc_gia', 'sáng_lập', 'tổ_chức', 'hoạt_động', 'xã_hội', 'dự_án', 'cung_cấp', 'áo_ấm', 'trẻ_em', 'vùng_cao', 'dự_án', 'phát_triển', 'túi', 'giấy', 'bảo_vệ', 'môi_trường', 'thành_tích', 'nổi_bật', 'vinh_dự', 'gương_mặt', 'trẻ', 'đại_diện', 'tham_dự', 'chương_trình', 'thủ_lĩnh', 'thanh_niên', 'ngoại_giao', 'trường', 'đại_học', 'tổ_chức', 'đại_diện', 'tham_dự', 'đối_thoại', 'giáo_dục', 'đặc_biệt', 'xuất_sắc', 'ứng_viên', 'thí_sinh', 'thế_giới', 'học_bổng', 'toàn_phần', 'trường', 'đại_học', '

In [13]:
df['text_length'] = df['Processed_Content'].apply(len)

In [14]:
print(df.head(10))

                                   Processed_Content  Category  text_length
0  [lễ, chủ_tịch, tỉnh, trao, khen, chủ_tịch, tỉn...  Giao duc           97
1  [đề_xuất, giáo_dục, kỳ_vọng, đột_phá, dư_luận,...  Giao duc          267
2  [gần, cư_dân, mạng, truyền, đoạn, clip, ghi, h...  Giao duc          172
3  [căn_bệnh, thành_tích, địa_phương, biến, thi, ...  Giao duc          370
4  [phân_biệt, dạy, tác_phẩm, chương_trình, ban, ...  Giao duc          325
5  [trường, năm_học, nộp, tiền, quản_lý, phép, dạ...  Giao duc          204
6  [công_nghệ, may, hút, thí_sinh, nữ, tranh, tuy...  Giao duc          225
7  [ảnh, minh_hoạ, phương_án, tổ_chức, quy_trình,...  Giao duc          213
8  [học_sinh, bậc, mầm_non, tiểu_học, nghỉ, học, ...  Giao duc          142
9  [thi, kỳ, thi, ghi_nhận, phóng_viên, báo_điện_...  Giao duc          412


In [15]:
# Tính độ dài của mỗi chuỗi trong sequences
lengths = [len(sentence) for sentence in df['Processed_Content']]

# Xác định giá trị max_length dựa trên phân phối của độ dài chuỗi
max_length = int(np.percentile(lengths, 95))
print(max_length)

# Tính trung bình của max_length
average_max_length = np.mean(lengths)
print(average_max_length)

392
162.2473102290023


## Convert label to one-hot-encoding

In [16]:
y = pd.get_dummies(labels)

## Tokenization

In [17]:
def txtTokenizer(texts):
    # Khởi tạo Tokenizer với số từ tối đa là 26313
    tokenizer = Tokenizer(num_words=26313)
    
    # Huấn luyện Tokenizer trên danh sách văn bản
    tokenizer.fit_on_texts(texts)

    # Lấy từ điển từ (word index) từ Tokenizer
    word_index = tokenizer.word_index

    return tokenizer, word_index


tokenizer, word_index = txtTokenizer(news)


In [18]:
# In ra word_index của một từ cụ thể
word_to_lookup = "cầu_thủ"

if word_to_lookup in word_index:
    print(f"Index of '{word_to_lookup}': {word_index[word_to_lookup]}")
else:
    print(f"'{word_to_lookup}' not found in vocabulary.")

count_of_word = 0

# Lặp qua dữ liệu để đếm số lần xuất hiện của từ
for sentence in news:
    if word_to_lookup in sentence:
        count_of_word += 1

print(f"The word '{word_to_lookup}' appears {count_of_word} times in the data.")

Index of 'cầu_thủ': 68
The word 'cầu_thủ' appears 6869 times in the data.


In [19]:
# Print the first 10 items from word_index
for index, (word, value) in enumerate(word_index.items()):
    print(f"{word}: {value}")
    
    # Print only the first 10 items
    if index == 9:
        break

đi: 1
trường: 2
thi: 3
học: 4
trẻ: 5
tiền: 6
đầu: 7
đội: 8
công_ty: 9
tổ_chức: 10


## Sequence and padding

In [20]:
# Chuyển đổi văn bản thành chuỗi các chỉ số tương ứng sử dụng tokenizer
news_token = tokenizer.texts_to_sequences(news)

In [21]:
print(news[1])
print(news_token[1])
print(len(news[1]))
print(len(news_token[1]))

['đề_xuất', 'giáo_dục', 'kỳ_vọng', 'đột_phá', 'dư_luận', 'theo', 'hàng_loạt', 'rút', 'ngắn', 'học', 'đào_tạo', 'hình_thức', 'tín_chỉ', 'học', 'trực_tuyến', 'công_nhận', 'tốt_nghiệp', 'tự_biên', 'soạn', 'nâng', 'chế_độ', 'lương', 'giáo_viên', 'văn_phòng', 'đề_xuất', 'giúp', 'học_sinh', 'hoàn_cảnh', 'tiếp_cận', 'giáo_dục', 'đại_học', 'đề_xuất', 'dư_luận', 'hình_thức', 'giáo_dục', 'đào_tạo', 'dạng', 'tín_chỉ', 'áp_dụng', 'đề_xuất', 'học_sinh', 'học', 'tín_chỉ', 'ảnh', 'tin', 'theo', 'lãnh_đạo', 'thành_phố', 'áp_dụng', 'tín_chỉ', 'liên_thông', 'bậc', 'đại_học', 'sau_đại_học', 'giúp', 'học_sinh', 'làm_quen', 'phương_thức', 'học', 'tín_chỉ', 'hiện_đại', 'tiết_kiệm', 'học', 'học', 'giỏi', 'đề_án', 'hướng', 'tiếp_cận', 'kiến_thức', 'phổ_thông', 'học', 'đại_học', 'học_sinh', 'đại_học', 'chờ', 'quy_định', 'thế_giới', 'quy_định', 'áp_dụng', 'thành_công', 'đề_án', 'thành_phố', 'dự_tính', 'năm_học', 'môn', 'bắt_buộc', 'môn', 'hoàn_thành', 'chủ_tịch', 'hội_đồng', 'chuyên_môn', 'trường', 'giảng_viên'

In [22]:
# Độ dài tối đa của mỗi chuỗi chỉ số
maxlen = 200

# Padding chuỗi chỉ số để có chiều dài tối đa là maxlen
news_pad = pad_sequences(news_token, padding='post', maxlen=maxlen)

In [23]:
print(news_pad[1])

[  867   369  1190     4    35    14    35   428    69    26    69   269
   121  1543    49  3516   516   128  1470   128   484   132   489   633
     2  1095     4  5339   212    69    14   465     1   646   428    14
   808   158   808  7645   138   465   228     4  5339   212  2957  1234
    16  2284   674    19   438   546    84  7646  2339  1383   752     4
    14     3  1820     2   117  2096   501   667   757    20   278   584
   178   293  5339    14   442  4728  1694  6105  1534  2134   336  1273
   487    14  2569   336   884   336  2339     1   116   487 17723   278
     4  5339    10   189   207    52    92  1504   727  1574  1473  2846
    28    28    40    35   757     4  5339  2915   757   438   546   516
  1752    16  9199  2038    96    40   757  2418    96    40    70   672
   582    20  1028    28    28    40    35     4  5339    14     4   646
 19391   516  1246     1  4125   209    49  2163    96    84  5339  1460
   418    48  3064     2    35   296    84  5339  3

## Split data

### Split

In [24]:
# Sử dụng train_test_split để chia dữ liệu thành train, test, val
X_train, X_temp, y_train, y_temp = train_test_split(news_pad, y, test_size=0.4, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
# Lưu dữ liệu vào file data.pkl
data = {
    'X_train': X_train,
    'y_train': y_train,
    'X_test': X_test,
    'y_test': y_test,
    'X_val': X_val,
    'y_val': y_val,
    'word_index': word_index,
    'tokenizer': tokenizer,
}

with open('data.pkl', 'wb') as file:
    pickle.dump(data, file)

### If already splitted

In [5]:
with open('data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)
X_train,y_train, X_test, y_test,X_val, y_val, word_index,tokenizer = loaded_data.values()

In [6]:
# In ra kích thước của từng tập
print("Kích thước tập train:", X_train.shape, y_train.shape)
print("Kích thước tập test:", X_test.shape, y_test.shape)
print("Kích thước tập validation:", X_val.shape, y_val.shape)

Kích thước tập train: (68593, 200) (68593, 12)
Kích thước tập test: (22864, 200) (22864, 12)
Kích thước tập validation: (22865, 200) (22865, 12)


In [39]:
counts_train = np.sum(y_train, axis=0)
print("Số lượng mẫu trên từng nhãn trong tập train:")
for label, count in zip(y_train.columns, counts_train):
    print(f"{label}: {count} mẫu")

counts_test = np.sum(y_test, axis=0)
print("\nSố lượng mẫu trên từng nhãn trong tập test:")
for label, count in zip(y_test.columns, counts_test):
    print(f"{label}: {count} mẫu")

counts_val = np.sum(y_val, axis=0)
print("\nSố lượng mẫu trên từng nhãn trong tập validation:")
for label, count in zip(y_val.columns, counts_val):
    print(f"{label}: {count} mẫu")

Số lượng mẫu trên từng nhãn trong tập train:
Chinh tri Xa hoi: 6463 mẫu
Cong nghe: 5140 mẫu
Doi song: 6106 mẫu
Giai tri: 5364 mẫu
Giao duc: 5542 mẫu
Khoa hoc: 6031 mẫu
Kinh doanh: 5665 mẫu
Phap luat: 5816 mẫu
Suc khoe: 5325 mẫu
The gioi: 5765 mẫu
The thao: 5889 mẫu
Van hoa: 5487 mẫu

Số lượng mẫu trên từng nhãn trong tập test:
Chinh tri Xa hoi: 2152 mẫu
Cong nghe: 1737 mẫu
Doi song: 2038 mẫu
Giai tri: 1825 mẫu
Giao duc: 1837 mẫu
Khoa hoc: 1959 mẫu
Kinh doanh: 1819 mẫu
Phap luat: 1918 mẫu
Suc khoe: 1694 mẫu
The gioi: 1877 mẫu
The thao: 2062 mẫu
Van hoa: 1946 mẫu

Số lượng mẫu trên từng nhãn trong tập validation:
Chinh tri Xa hoi: 2171 mẫu
Cong nghe: 1610 mẫu
Doi song: 2051 mẫu
Giai tri: 1834 mẫu
Giao duc: 1842 mẫu
Khoa hoc: 1926 mẫu
Kinh doanh: 1844 mẫu
Phap luat: 1922 mẫu
Suc khoe: 1782 mẫu
The gioi: 1972 mẫu
The thao: 2014 mẫu
Van hoa: 1897 mẫu


## Embedding matrix 

### Word2Vec

In [33]:

# In thông báo "Load Word2Vec..."
print("Load Word2Vec...")

# Load mô hình Word2Vec từ đường dẫn đã chỉ định
word2vec_model = Word2Vec.load(MODEL_FOLDER + sep + WORD2VEC_MODEL)

# Lấy kích thước của vector embedding
embedding_dim = word2vec_model.wv.vector_size 
print("Embedding dim: ", embedding_dim)

# Xác định số lượng từ trong từ điển, không vượt quá số lượng từ trong mô hình Word2Vec
num_words = min(len(word_index) + 1, len(word2vec_model.wv.vocab) + 1)
print("Vocabulary size: ", num_words)

# Khởi tạo ma trận embedding với tất cả giá trị là 0
embedding_matrix_word2vec = np.zeros((num_words, embedding_dim))

# Lặp qua từ điển từ và cập nhật ma trận embedding nếu từ đó có trong mô hình Word2Vec
for word, i in word_index.items():
    if i >= num_words:
        continue
    if word in word2vec_model.wv:
        embedding_matrix_word2vec[i] = word2vec_model.wv[word]

# In thông báo kiểm tra embedding_matrix
print("Shape of the embedding matrix:", embedding_matrix_word2vec.shape)

Load Word2Vec...


Embedding dim:  100
Vocabulary size:  26313
Shape of the embedding matrix: (26313, 100)


### Doc2Vec

In [9]:
# In thông báo "Load Doc2Vec..."
print("Load Doc2Vec...")

# Load mô hình Word2Vec từ đường dẫn đã chỉ định
doc2vec_model = Doc2Vec.load(MODEL_FOLDER + sep + DOC2VEC_MODEL)

# Lấy kích thước của vector embedding
embedding_dim = doc2vec_model.wv.vector_size 
print("Embedding dim: ", embedding_dim)

# Xác định số lượng từ trong từ điển, không vượt quá số lượng từ trong mô hình Doc2Vec
num_words = min(len(word_index) + 1, len(doc2vec_model.wv.vocab) + 1)
print("Vocabulary size: ", num_words)

# Khởi tạo ma trận embedding với tất cả giá trị là 0
embedding_matrix_doc2vec = np.zeros((num_words, embedding_dim))

# Lặp qua từ điển từ và cập nhật ma trận embedding nếu từ đó có trong mô hình Doc2Vec
for word, i in word_index.items():
    if i >= num_words:
        continue
    if word in doc2vec_model.wv:
        embedding_matrix_doc2vec[i] = doc2vec_model.wv[word]

# In thông báo kiểm tra embedding_matrix
print("Shape of the embedding matrix:", embedding_matrix_doc2vec.shape)

Load Doc2Vec...
Embedding dim:  100
Vocabulary size:  26313
Shape of the embedding matrix: (26313, 100)


## Build model

#### BiLSTM

In [35]:
def build_bilstm_model(lstm_units_1, lstm_units_2, dense_units, dropout_rate, learning_rate, embedding_matrix):
    model = Sequential()
    # Lớp Embedding với ma trận embedding đã được truyền vào
    model.add(Embedding(input_dim=num_words, 
                        output_dim=embedding_dim, 
                        input_length=maxlen, 
                        weights=[embedding_matrix]))
    # Lớp LSTM 1 với dropout
    model.add(Bidirectional(LSTM(lstm_units_1, return_sequences=True)))
    model.add(Dropout(dropout_rate))
    # Lớp LSTM 2 với dropout
    model.add(Bidirectional(LSTM(lstm_units_2, return_sequences=False)))
    model.add(Dropout(dropout_rate))
    # Lớp Dense với dropout và kích hoạt ReLU
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    # Lớp Dense cuối cùng với kích hoạt softmax
    model.add(Dense(12, activation='softmax'))
    # Sử dụng tối ưu hóa Adam với learning rate được truyền vào
    optimizer = Adam(learning_rate=learning_rate)
    # Biên soạn mô hình
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

#### LSTM

In [ ]:
def build_lstm_model(lstm_units_1, lstm_units_2, dense_units, dropout_rate, learning_rate, embedding_matrix):
    model = Sequential()
    # Lớp Embedding với ma trận embedding đã được truyền vào
    model.add(Embedding(input_dim=num_words, 
                        output_dim=embedding_dim, 
                        input_length=maxlen, 
                        weights=[embedding_matrix]))
    # Lớp LSTM 1 với dropout
    model.add(LSTM(lstm_units_1, return_sequences=True))
    model.add(Dropout(dropout_rate))
    # Lớp LSTM 2 với dropout
    model.add(LSTM(lstm_units_2, return_sequences=False))
    model.add(Dropout(dropout_rate))
    # Lớp Dense với dropout và kích hoạt ReLU
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    # Lớp Dense cuối cùng với kích hoạt softmax
    model.add(Dense(12, activation='softmax'))
    # Sử dụng tối ưu hóa Adam với learning rate được truyền vào
    optimizer = Adam(learning_rate=learning_rate)
    
    # Biên soạn mô hình
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


#### GRU

In [6]:
def build_gru_model(gru_units_1, gru_units_2, dense_units, dropout_rate, learning_rate, embedding_matrix):
    model = Sequential()
    # Lớp Embedding với ma trận embedding đã được truyền vào
    model.add(Embedding(input_dim=num_words, 
                        output_dim=embedding_dim, 
                        input_length=maxlen, 
                        weights=[embedding_matrix]))
    # Lớp LSTM 1 với dropout
    model.add(GRU(gru_units_1, return_sequences=True))
    model.add(Dropout(dropout_rate))
    # Lớp LSTM 2 với dropout
    model.add(GRU(gru_units_2, return_sequences=False))
    model.add(Dropout(dropout_rate))
    # Lớp Dense với dropout và kích hoạt ReLU
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    # Lớp Dense cuối cùng với kích hoạt softmax
    model.add(Dense(12, activation='softmax'))
    # Sử dụng tối ưu hóa Adam với learning rate được truyền vào
    optimizer = Adam(learning_rate=learning_rate)
    # Biên soạn mô hình
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

## Finding hyperparameters

#### BiLSTM

In [36]:
def objective_bilstm(trial, embedding_matrix):
    lstm_units_1 = trial.suggest_int('lstm_units_1', 64, 512, step=32)
    lstm_units_2 = trial.suggest_int('lstm_units_2', lstm_units_1//2, lstm_units_1, step=32)
    dense_units = trial.suggest_int('dense_units', 64, 512, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5, step=0.1)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    epochs = 10
    batch_size = 256

    print(f"Trying hyperparameters: lstm_units_1={lstm_units_1}, lstm_units_2={lstm_units_2}, dense_units={dense_units}, "
          f"dropout_rate={dropout_rate}, learning_rate={learning_rate}, batch_size={batch_size}")

    model = build_bilstm_model(lstm_units_1, lstm_units_2, dense_units, dropout_rate, learning_rate, embedding_matrix)

    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
              validation_data=(X_val, y_val), callbacks=[TFKerasPruningCallback(trial, "val_loss")], verbose=1)

    _, accuracy = model.evaluate(X_test, y_test, verbose=0)

    return accuracy

#### GRU

In [7]:
def objective_gru(trial, embedding_matrix ):
    gru_units_1 = trial.suggest_int('gru_units_1', 64, 512, step=32)
    gru_units_2 = trial.suggest_int('gru_units_2', 64, 512, step=32)
    dense_units = trial.suggest_int('dense_units', 64, 512, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5, step=0.1)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    epochs = 10
    batch_size = 256

    print(f"Trying hyperparameters: gru_units_1={gru_units_1}, gru_units_2={gru_units_2}, dense_units={dense_units}, "
          f"dropout_rate={dropout_rate}, learning_rate={learning_rate}, batch_size={batch_size}")

    model = build_gru_model(gru_units_1, gru_units_2, dense_units, dropout_rate, learning_rate, embedding_matrix)

    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
              validation_data=(X_val, y_val), callbacks=[TFKerasPruningCallback(trial, "val_loss")], verbose=1)

    _, accuracy = model.evaluate(X_test, y_test, verbose=0)

    return accuracy

#### LSTM

In [ ]:
def objective_lstm(trial, embedding_matrix):
    lstm_units_1 = trial.suggest_int('lstm_units_1', 64, 512, step=32)
    lstm_units_2 = trial.suggest_int('lstm_units_2', lstm_units_1//2, lstm_units_1, step=32)
    dense_units = trial.suggest_int('dense_units', 64, 512, step=32)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5, step=0.1)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    epochs = 20
    batch_size = 256

    print(f"Trying hyperparameters: lstm_units_1={lstm_units_1}, lstm_units_2={lstm_units_2}, dense_units={dense_units}, "
          f"dropout_rate={dropout_rate}, learning_rate={learning_rate}, batch_size={batch_size}")

    model = build_lstm_model(lstm_units_1, lstm_units_2, dense_units, dropout_rate, learning_rate, embedding_matrix)

    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
              validation_data=(X_val, y_val), callbacks=[TFKerasPruningCallback(trial, "val_loss")], verbose=1)

    _, accuracy = model.evaluate(X_test, y_test, verbose=0)

    return accuracy
    

## Create study and optimize 

#### with Word2Vec

In [ ]:
embedding_matrix = embedding_matrix_word2vec

In [ ]:
study_lstm_word2vec = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study_lstm_word2vec.optimize(lambda trial: objective_lstm(trial, embedding_matrix), n_trials=50)

Trying hyperparameters: lstm_units_1=160, lstm_units_2=144, dense_units=192, dropout_rate=0.2, learning_rate=0.0009390052419240196, batch_size=256


2023-12-10 18:01:21.851330: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-12-10 18:01:21.853018: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-12-10 18:01:24.722794: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:88:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2023-12-10 18:01:24.723560: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:89:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2023-12-10 18:01:24.724293: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:b1:00.0 name: Tesla T4 computeCapability: 

Epoch 1/15


2023-12-10 18:01:30.684235: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-12-10 18:01:31.009533: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


268/268 [==============================] - 26s 86ms/step - loss: 1.8042 - accuracy: 0.3529 - val_loss: 1.4355 - val_accuracy: 0.4984
Epoch 2/15
268/268 [==============================] - 22s 83ms/step - loss: 1.6632 - accuracy: 0.4139 - val_loss: 1.4792 - val_accuracy: 0.4830
Epoch 3/15
268/268 [==============================] - 22s 83ms/step - loss: 1.4294 - accuracy: 0.5038 - val_loss: 1.2048 - val_accuracy: 0.5882
Epoch 4/15
268/268 [==============================] - 22s 82ms/step - loss: 1.2582 - accuracy: 0.5661 - val_loss: 1.0259 - val_accuracy: 0.6489
Epoch 5/15
268/268 [==============================] - 22s 83ms/step - loss: 1.0605 - accuracy: 0.6379 - val_loss: 0.9743 - val_accuracy: 0.6620
Epoch 6/15
268/268 [==============================] - 22s 83ms/step - loss: 1.0286 - accuracy: 0.6442 - val_loss: 1.1194 - val_accuracy: 0.5981
Epoch 7/15
268/268 [==============================] - 22s 83ms/step - loss: 1.0529 - accuracy: 0.6333 - val_loss: 1.0307 - val_accuracy: 0.6438
Epo

In [ ]:
embedding_matrix = embedding_matrix_word2vec
study_bilstm_word2vec = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study_bilstm_word2vec.optimize(lambda trial: objective_bilstm(trial, embedding_matrix), n_trials=50)

Trying hyperparameters: lstm_units_1=256, lstm_units_2=256, dense_units=160, dropout_rate=0.2, learning_rate=0.00016188936476430271, batch_size=256
Epoch 1/20
268/268 [==============================] - 39s 137ms/step - loss: 1.9695 - accuracy: 0.3091 - val_loss: 1.3654 - val_accuracy: 0.4905
Epoch 2/20
268/268 [==============================] - 36s 135ms/step - loss: 1.6126 - accuracy: 0.4302 - val_loss: 1.4670 - val_accuracy: 0.5073
Epoch 3/20
268/268 [==============================] - 37s 136ms/step - loss: 1.3920 - accuracy: 0.5213 - val_loss: 1.3228 - val_accuracy: 0.5473
Epoch 4/20
268/268 [==============================] - 37s 138ms/step - loss: 1.2954 - accuracy: 0.5460 - val_loss: 1.1367 - val_accuracy: 0.6047
Epoch 5/20
268/268 [==============================] - 37s 139ms/step - loss: 1.1680 - accuracy: 0.5962 - val_loss: 1.0648 - val_accuracy: 0.6362
Epoch 6/20
268/268 [==============================] - 37s 139ms/step - loss: 1.0759 - accuracy: 0.6338 - val_loss: 0.9795 - val

In [ ]:
embedding_matrix = embedding_matrix_word2vec
study_gru_word2vec = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study_gru_word2vec.optimize(lambda trial: objective_gru(trial, embedding_matrix), n_trials=50)

Trying hyperparameters: gru_units_1=384, gru_units_2=64, dense_units=288, dropout_rate=0.2, learning_rate=0.002134662130371001, batch_size=256
Epoch 1/10
268/268 [==============================] - 32s 110ms/step - loss: 1.6722 - accuracy: 0.4063 - val_loss: 0.4949 - val_accuracy: 0.8304
Epoch 2/10
268/268 [==============================] - 29s 109ms/step - loss: 0.4390 - accuracy: 0.8497 - val_loss: 0.4107 - val_accuracy: 0.8605
Epoch 3/10
268/268 [==============================] - 29s 110ms/step - loss: 0.2988 - accuracy: 0.8998 - val_loss: 0.3880 - val_accuracy: 0.8712
Epoch 4/10
268/268 [==============================] - 30s 111ms/step - loss: 0.2109 - accuracy: 0.9297 - val_loss: 0.4093 - val_accuracy: 0.8732
Epoch 5/10
268/268 [==============================] - 30s 112ms/step - loss: 0.1423 - accuracy: 0.9539 - val_loss: 0.5412 - val_accuracy: 0.8299
Epoch 6/10
268/268 [==============================] - 30s 112ms/step - loss: 0.2195 - accuracy: 0.9255 - val_loss: 0.4471 - val_accu

#### with Doc2Vec

In [10]:
embedding_matrix = embedding_matrix_doc2vec
maxlen=200

In [ ]:
study_lstm_doc2vec = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study_lstm_doc2vec.optimize(lambda trial: objective_lstm(trial, embedding_matrix), n_trials=50)

In [37]:
study_bilstm_doc2vec = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study_bilstm_doc2vec.optimize(lambda trial: objective_bilstm(trial, embedding_matrix), n_trials=50)

Trying hyperparameters: lstm_units_1=288, lstm_units_2=240, dense_units=320, dropout_rate=0.2, learning_rate=0.003852952188934468, batch_size=256


2023-12-13 01:18:36.012470: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-12-13 01:18:36.014648: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-12-13 01:18:36.064084: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:88:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2023-12-13 01:18:36.064979: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:89:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2023-12-13 01:18:36.065820: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:b1:00.0 name: Tesla T4 computeCapability: 

Epoch 1/10


2023-12-13 01:18:44.198213: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-12-13 01:18:44.508741: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


268/268 [==============================] - 102s 357ms/step - loss: 0.9626 - accuracy: 0.6803 - val_loss: 0.6065 - val_accuracy: 0.8045
Epoch 2/10
268/268 [==============================] - 97s 363ms/step - loss: 0.5712 - accuracy: 0.8171 - val_loss: 0.5144 - val_accuracy: 0.8311
Epoch 3/10
268/268 [==============================] - 99s 369ms/step - loss: 0.4249 - accuracy: 0.8616 - val_loss: 0.4368 - val_accuracy: 0.8575
Epoch 4/10
268/268 [==============================] - 99s 371ms/step - loss: 0.3345 - accuracy: 0.8905 - val_loss: 0.4455 - val_accuracy: 0.8577
Epoch 5/10
268/268 [==============================] - 100s 373ms/step - loss: 0.2665 - accuracy: 0.9133 - val_loss: 0.4426 - val_accuracy: 0.8668
Epoch 6/10
268/268 [==============================] - 100s 373ms/step - loss: 0.2012 - accuracy: 0.9344 - val_loss: 0.4447 - val_accuracy: 0.8715
Epoch 7/10
268/268 [==============================] - 100s 373ms/step - loss: 0.1529 - accuracy: 0.9503 - val_loss: 0.4979 - val_accuracy:

In [53]:
study_gru_doc2vec = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study_gru_doc2vec.optimize(lambda trial: objective_gru(trial, embedding_matrix), n_trials=50)

Trying hyperparameters: gru_units_1=448, gru_units_2=128, dense_units=320, dropout_rate=0.2, learning_rate=0.0003678798568175657, batch_size=256


Epoch 1/10
268/268 [==============================] - 45s 160ms/step - loss: 1.9606 - accuracy: 0.2933 - val_loss: 1.2728 - val_accuracy: 0.5978
Epoch 2/10
268/268 [==============================] - 43s 159ms/step - loss: 1.0705 - accuracy: 0.6370 - val_loss: 0.6938 - val_accuracy: 0.7569
Epoch 3/10
268/268 [==============================] - 43s 161ms/step - loss: 0.6733 - accuracy: 0.7669 - val_loss: 0.6907 - val_accuracy: 0.7630
Epoch 4/10
268/268 [==============================] - 43s 160ms/step - loss: 0.6248 - accuracy: 0.7858 - val_loss: 0.5469 - val_accuracy: 0.8099
Epoch 5/10
268/268 [==============================] - 43s 162ms/step - loss: 0.5121 - accuracy: 0.8232 - val_loss: 0.4830 - val_accuracy: 0.8316
Epoch 6/10
268/268 [==============================] - 43s 162ms/step - loss: 0.4670 - accuracy: 0.8380 - val_loss: 0.4748 - val_accuracy: 0.8356
Epoch 7/10
268/268 [==============================] - 44s 163ms/step - loss: 0.4285 - accuracy: 0.8512 - val_loss: 0.4400 - val_ac

## Get best parameters 

### Save hyperparameters

#### Word2Vec

In [ ]:
# Word2Vec
complete_lstm_word2vec_trials = study_lstm_word2vec.trials_dataframe()[study_lstm_word2vec.trials_dataframe()['state'] == 'COMPLETE']
complete_lstm_word2vec_trials.to_csv("./trials_result/study_lstm_word2vec_trials.csv", index=False)

In [ ]:
complete_bilstm_word2vec_trials = study_bilstm_word2vec.trials_dataframe()[study_bilstm_word2vec.trials_dataframe()['state'] == 'COMPLETE']
complete_bilstm_word2vec_trials.to_csv("./trials_result/study_bilstm_word2vec_trials.csv", index=False)

In [ ]:
complete_trials_word2vec_gru = study_gru_word2vec.trials_dataframe()[study_gru_word2vec.trials_dataframe()['state'] == 'COMPLETE']
complete_trials_word2vec_gru.to_csv("./trials_result/study_gru_word2vec_trials.csv", index=False)

In [ ]:
# Word2Vec
best_hyperparameters_lstm_word2vec = {
    'lstm_units_1': study_lstm_word2vec.best_trial.params['lstm_units_1'],
    'lstm_units_2': study_lstm_word2vec.best_trial.params['lstm_units_2'],
    'dense_units': study_lstm_word2vec.best_trial.params['dense_units'],
    'dropout_rate': study_lstm_word2vec.best_trial.params['dropout_rate'],
    'learning_rate': study_lstm_word2vec.best_trial.params['learning_rate']
    
}
# Save the best hyperparameters to a JSON file
with open('./hyperparameters/LSTM_Word2Vec.json', 'w') as file:
    json.dump(best_hyperparameters_lstm_word2vec, file)



In [ ]:
best_hyperparameters_bilstm_word2vec = {
    'lstm_units_1': study_bilstm_word2vec.best_trial.params['lstm_units_1'],
    'lstm_units_2': study_bilstm_word2vec.best_trial.params['lstm_units_2'],
    'dense_units': study_bilstm_word2vec.best_trial.params['dense_units'],
    'dropout_rate': study_bilstm_word2vec.best_trial.params['dropout_rate'],
    'learning_rate': study_bilstm_word2vec.best_trial.params['learning_rate']
    
}
# Save the best hyperparameters to a JSON file
with open('./hyperparameters/BiLSTM_Word2Vec.json', 'w') as file:
    json.dump(best_hyperparameters_bilstm_word2vec, file)

In [ ]:
best_hyperparameters_gru_word2vec = {
    'gru_units_1': study_gru_word2vec.best_trial.params['gru_units_1'],
    'gru_units_2': study_gru_word2vec.best_trial.params['gru_units_2'],
    'dense_units': study_gru_word2vec.best_trial.params['dense_units'],
    'dropout_rate': study_gru_word2vec.best_trial.params['dropout_rate'],
    'learning_rate': study_gru_word2vec.best_trial.params['learning_rate']
    
}
# Save the best hyperparameters to a JSON file
with open('./hyperparameters/GRU_Word2Vec.json', 'w') as file:
    json.dump(best_hyperparameters_gru_word2vec, file)

#### Doc2Vec

In [ ]:
# Doc2Vec
complete_trials_doc2vec_lstm = study_lstm_doc2vec.trials_dataframe()[study_lstm_doc2vec.trials_dataframe()['state'] == 'COMPLETE']
complete_trials_doc2vec_lstm.to_csv("./trials_result/study_lstm_doc2vec.csv", index=False)

In [38]:
complete_trials_doc2vec_bilstm = study_bilstm_doc2vec.trials_dataframe()[study_bilstm_doc2vec.trials_dataframe()['state'] == 'COMPLETE']
complete_trials_doc2vec_bilstm.to_csv("./trials_result/study_bilstm_doc2vec.csv", index=False)

In [54]:
complete_trials_doc2vec_gru = study_gru_doc2vec.trials_dataframe()[study_gru_doc2vec.trials_dataframe()['state'] == 'COMPLETE']
complete_trials_doc2vec_gru.to_csv("./trials_result/study_gru_doc2vec.csv", index=False)

In [ ]:
# Doc2Vec
best_hyperparameters_lstm_doc2vec = {
    'lstm_units_1': study_lstm_doc2vec.best_trial.params['lstm_units_1'],
    'lstm_units_2': study_lstm_doc2vec.best_trial.params['lstm_units_2'],
    'dense_units': study_lstm_doc2vec.best_trial.params['dense_units'],
    'dropout_rate': study_lstm_doc2vec.best_trial.params['dropout_rate'],
    'learning_rate': study_lstm_doc2vec.best_trial.params['learning_rate']
    
}
# Save the best hyperparameters to a JSON file
with open('./hyperparameters/LSTM_Doc2Vec.json', 'w') as file:
    json.dump(best_hyperparameters_lstm_doc2vec, file)


In [40]:
best_hyperparameters_bilstm_doc2vec = {
    'lstm_units_1': study_bilstm_doc2vec.best_trial.params['lstm_units_1'],
    'lstm_units_2': study_bilstm_doc2vec.best_trial.params['lstm_units_2'],
    'dense_units': study_bilstm_doc2vec.best_trial.params['dense_units'],
    'dropout_rate': study_bilstm_doc2vec.best_trial.params['dropout_rate'],
    'learning_rate': study_bilstm_doc2vec.best_trial.params['learning_rate']
    
}
# Save the best hyperparameters to a JSON file
with open('./hyperparameters/BiLSTM_Doc2Vec.json', 'w') as file:
    json.dump(best_hyperparameters_bilstm_doc2vec, file)

In [56]:
best_hyperparameters_gru_doc2vec = {
    'gru_units_1': study_gru_doc2vec.best_trial.params['gru_units_1'],
    'gru_units_2': study_gru_doc2vec.best_trial.params['gru_units_2'],
    'dense_units': study_gru_doc2vec.best_trial.params['dense_units'],
    'dropout_rate': study_gru_doc2vec.best_trial.params['dropout_rate'],
    'learning_rate': study_gru_doc2vec.best_trial.params['learning_rate']
    
}
# Save the best hyperparameters to a JSON file
with open('./hyperparameters/GRU_Doc2Vec.json', 'w') as file:
    json.dump(best_hyperparameters_gru_doc2vec, file)

### Training with best hyperparameters

#### Word2Vec

In [ ]:
# Word2Vec
best_params_lstm_word2vec = study_lstm_word2vec.best_params

# Create a new dictionary excluding 'epochs' and 'batch_size'
filtered_params = {key: value for key, value in best_params_lstm_word2vec.items()}
filtered_params['embedding_matrix'] = embedding_matrix_word2vec
# Now, filtered_params contains all parameters from best_params_bilstm_word2vec except 'epochs' and 'batch_size'
best_model_lstm_word2vec = build_lstm_model(**filtered_params)
best_model_lstm_word2vec.fit(X_train, y_train, epochs=15, batch_size=256, validation_data=(X_val, y_val))

Epoch 1/15


268/268 [==============================] - 35s 116ms/step - loss: 1.8528 - accuracy: 0.3367 - val_loss: 1.3665 - val_accuracy: 0.4916
Epoch 2/15
268/268 [==============================] - 31s 116ms/step - loss: 1.3931 - accuracy: 0.5112 - val_loss: 1.1729 - val_accuracy: 0.5823
Epoch 3/15
268/268 [==============================] - 31s 116ms/step - loss: 1.1659 - accuracy: 0.5921 - val_loss: 0.9759 - val_accuracy: 0.6402
Epoch 4/15
268/268 [==============================] - 31s 117ms/step - loss: 0.9734 - accuracy: 0.6586 - val_loss: 0.7353 - val_accuracy: 0.7332
Epoch 5/15
268/268 [==============================] - 32s 118ms/step - loss: 0.7395 - accuracy: 0.7348 - val_loss: 0.6450 - val_accuracy: 0.7705
Epoch 6/15
268/268 [==============================] - 32s 119ms/step - loss: 0.6243 - accuracy: 0.7841 - val_loss: 0.5662 - val_accuracy: 0.8072
Epoch 7/15
268/268 [==============================] - 32s 119ms/step - loss: 0.5439 - accuracy: 0.8170 - val_loss: 0.5378 - val_accuracy: 0.8

In [ ]:
best_params_bilstm_word2vec = study_bilstm_word2vec.best_params

# Create a new dictionary excluding 'epochs' and 'batch_size'
filtered_params = {key: value for key, value in best_params_bilstm_word2vec.items()}
filtered_params['embedding_matrix'] = embedding_matrix_word2vec
# Now, filtered_params contains all parameters from best_params_bilstm_word2vec except 'epochs' and 'batch_size'
best_model_bilstm_word2vec = build_bilstm_model(**filtered_params)
best_model_bilstm_word2vec.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_val, y_val))

Epoch 1/10
268/268 [==============================] - 154s 561ms/step - loss: 0.9042 - accuracy: 0.7025 - val_loss: 0.5830 - val_accuracy: 0.8033
Epoch 2/10
268/268 [==============================] - 149s 558ms/step - loss: 0.5390 - accuracy: 0.8189 - val_loss: 0.4865 - val_accuracy: 0.8354
Epoch 3/10
268/268 [==============================] - 150s 560ms/step - loss: 0.4496 - accuracy: 0.8479 - val_loss: 0.4549 - val_accuracy: 0.8471
Epoch 4/10
268/268 [==============================] - 150s 559ms/step - loss: 0.3815 - accuracy: 0.8706 - val_loss: 0.4189 - val_accuracy: 0.8579
Epoch 5/10
268/268 [==============================] - 150s 559ms/step - loss: 0.3232 - accuracy: 0.8907 - val_loss: 0.4225 - val_accuracy: 0.8559
Epoch 6/10
268/268 [==============================] - 150s 560ms/step - loss: 0.2853 - accuracy: 0.9026 - val_loss: 0.3992 - val_accuracy: 0.8702
Epoch 7/10
268/268 [==============================] - 150s 560ms/step - loss: 0.2225 - accuracy: 0.9253 - val_loss: 0.4123 -

In [ ]:
best_params_gru_word2vec = study_gru_word2vec.best_params

# Create a new dictionary excluding 'epochs' and 'batch_size'
filtered_params = {key: value for key, value in best_params_gru_word2vec.items()}
filtered_params['embedding_matrix'] = embedding_matrix_word2vec
# Now, filtered_params contains all parameters from best_params_bilstm_word2vec except 'epochs' and 'batch_size'
best_model_gru_word2vec = build_gru_model(**filtered_params)
best_model_gru_word2vec.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_val, y_val))

Epoch 1/10
268/268 [==============================] - 62s 223ms/step - loss: 1.7862 - accuracy: 0.3629 - val_loss: 0.6811 - val_accuracy: 0.7640
Epoch 2/10
268/268 [==============================] - 60s 225ms/step - loss: 0.6470 - accuracy: 0.7802 - val_loss: 0.5466 - val_accuracy: 0.8136
Epoch 3/10
268/268 [==============================] - 61s 228ms/step - loss: 0.5508 - accuracy: 0.8135 - val_loss: 0.4892 - val_accuracy: 0.8338
Epoch 4/10
268/268 [==============================] - 62s 231ms/step - loss: 0.4454 - accuracy: 0.8471 - val_loss: 0.4368 - val_accuracy: 0.8524
Epoch 5/10
268/268 [==============================] - 62s 232ms/step - loss: 0.3781 - accuracy: 0.8715 - val_loss: 0.4362 - val_accuracy: 0.8539
Epoch 6/10
268/268 [==============================] - 62s 233ms/step - loss: 0.3351 - accuracy: 0.8861 - val_loss: 0.4264 - val_accuracy: 0.8585
Epoch 7/10
268/268 [==============================] - 63s 234ms/step - loss: 0.2880 - accuracy: 0.9041 - val_loss: 0.4017 - val_ac

#### Doc2Vec

In [ ]:
# Doc2Vec
# After optimization is complete, get the best params
best_params_lstm_doc2vec = study_lstm_doc2vec.best_params

# Create a new dictionary excluding 'epochs' and 'batch_size'
filtered_params = {key: value for key, value in best_params_lstm_doc2vec.items()}
filtered_params['embedding_matrix'] = embedding_matrix_doc2vec

# Now, filtered_params contains all parameters from best_params_lstm_doc2vec except 'epochs' and 'batch_size'
best_model_lstm_doc2vec = build_lstm_model(**filtered_params)

# Fit the model with the EarlyStopping callback
best_model_lstm_doc2vec.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=256,
    validation_data=(X_val, y_val)
)

Epoch 1/10
268/268 [==============================] - 37s 131ms/step - loss: 2.0326 - accuracy: 0.2895 - val_loss: 1.0692 - val_accuracy: 0.6364
Epoch 2/10
268/268 [==============================] - 35s 130ms/step - loss: 0.9910 - accuracy: 0.6520 - val_loss: 0.6715 - val_accuracy: 0.7569
Epoch 3/10
268/268 [==============================] - 35s 131ms/step - loss: 0.6717 - accuracy: 0.7703 - val_loss: 0.5346 - val_accuracy: 0.8208
Epoch 4/10
268/268 [==============================] - 35s 131ms/step - loss: 0.5085 - accuracy: 0.8331 - val_loss: 0.4830 - val_accuracy: 0.8396
Epoch 5/10
268/268 [==============================] - 35s 132ms/step - loss: 0.4171 - accuracy: 0.8651 - val_loss: 0.4441 - val_accuracy: 0.8524
Epoch 6/10
268/268 [==============================] - 36s 133ms/step - loss: 0.3409 - accuracy: 0.8925 - val_loss: 0.4362 - val_accuracy: 0.8625
Epoch 7/10
268/268 [==============================] - 35s 132ms/step - loss: 0.2659 - accuracy: 0.9168 - val_loss: 0.4289 - val_ac

In [49]:
best_params_bilstm_doc2vec = study_bilstm_doc2vec.best_params

# Create a new dictionary excluding 'epochs' and 'batch_size'
filtered_params = {key: value for key, value in best_params_bilstm_doc2vec.items()}
filtered_params['embedding_matrix'] = embedding_matrix_doc2vec

# Now, filtered_params contains all parameters from best_params_lstm_doc2vec except 'epochs' and 'batch_size'
best_model_bilstm_doc2vec = build_bilstm_model(**filtered_params)

# Fit the model with the EarlyStopping callback
best_model_bilstm_doc2vec.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=256,
    validation_data=(X_val, y_val)
)

Epoch 1/10
268/268 [==============================] - 28s 90ms/step - loss: 1.0989 - accuracy: 0.6381 - val_loss: 0.6039 - val_accuracy: 0.7979
Epoch 2/10
268/268 [==============================] - 23s 86ms/step - loss: 0.5725 - accuracy: 0.8123 - val_loss: 0.5060 - val_accuracy: 0.8342
Epoch 3/10
268/268 [==============================] - 23s 86ms/step - loss: 0.4621 - accuracy: 0.8514 - val_loss: 0.4696 - val_accuracy: 0.8498
Epoch 4/10
268/268 [==============================] - 23s 87ms/step - loss: 0.3797 - accuracy: 0.8790 - val_loss: 0.4548 - val_accuracy: 0.8519
Epoch 5/10
268/268 [==============================] - 23s 87ms/step - loss: 0.3093 - accuracy: 0.9009 - val_loss: 0.4312 - val_accuracy: 0.8666
Epoch 6/10
268/268 [==============================] - 23s 86ms/step - loss: 0.2503 - accuracy: 0.9197 - val_loss: 0.4335 - val_accuracy: 0.8694
Epoch 7/10
268/268 [==============================] - 23s 86ms/step - loss: 0.2022 - accuracy: 0.9360 - val_loss: 0.4389 - val_accuracy:

In [13]:
best_params_gru_doc2vec = study_gru_doc2vec.best_params

# Create a new dictionary excluding 'epochs' and 'batch_size'
filtered_params = {key: value for key, value in best_params_gru_doc2vec.items()}
filtered_params['embedding_matrix'] = embedding_matrix_doc2vec

# Now, filtered_params contains all parameters from best_params_lstm_doc2vec except 'epochs' and 'batch_size'
best_model_gru_doc2vec = build_gru_model(**filtered_params)

# Fit the model with the EarlyStopping callback
best_model_gru_doc2vec.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=256,
    validation_data=(X_val, y_val)
)

2023-12-14 04:30:12.554070: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-12-14 04:30:12.555435: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-12-14 04:30:15.403404: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:88:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2023-12-14 04:30:15.404217: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:89:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2023-12-14 04:30:15.404971: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:b1:00.0 name: Tesla T4 computeCapability: 

Epoch 1/10


2023-12-14 04:30:20.172773: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-12-14 04:30:20.482010: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


268/268 [==============================] - 46s 159ms/step - loss: 2.0284 - accuracy: 0.2663 - val_loss: 0.9289 - val_accuracy: 0.6844
Epoch 2/10
268/268 [==============================] - 42s 158ms/step - loss: 0.8661 - accuracy: 0.6961 - val_loss: 0.7168 - val_accuracy: 0.7466
Epoch 3/10
268/268 [==============================] - 43s 159ms/step - loss: 0.7019 - accuracy: 0.7513 - val_loss: 0.6182 - val_accuracy: 0.7805
Epoch 4/10
268/268 [==============================] - 43s 159ms/step - loss: 0.5688 - accuracy: 0.8011 - val_loss: 0.5225 - val_accuracy: 0.8182
Epoch 5/10
268/268 [==============================] - 43s 160ms/step - loss: 0.4947 - accuracy: 0.8277 - val_loss: 0.4863 - val_accuracy: 0.8314
Epoch 6/10
268/268 [==============================] - 43s 161ms/step - loss: 0.4680 - accuracy: 0.8377 - val_loss: 0.4617 - val_accuracy: 0.8375
Epoch 7/10
268/268 [==============================] - 43s 162ms/step - loss: 0.4231 - accuracy: 0.8514 - val_loss: 0.4433 - val_accuracy: 0.8

### Evaluate model

In [ ]:
best_model_gru_word2vec.evaluate(X_test,y_test)

715/715 [==============================] - 12s 16ms/step - loss: 0.4438 - accuracy: 0.8729


[0.4437906742095947, 0.8729443550109863]

In [ ]:
best_model_bilstm_word2vec.evaluate(X_test,y_test)

715/715 [==============================] - 24s 33ms/step - loss: 0.5025 - accuracy: 0.8704


[0.5025023818016052, 0.8704076409339905]

In [ ]:
best_model_lstm_word2vec.evaluate(X_test,y_test)

715/715 [==============================] - 12s 17ms/step - loss: 0.4951 - accuracy: 0.8636


[0.49508681893348694, 0.8636283874511719]

In [ ]:
best_model_lstm_doc2vec.evaluate(X_test,y_test)

  7/715 [..............................] - ETA: 14s - loss: 0.6645 - accuracy: 0.8348

715/715 [==============================] - 12s 17ms/step - loss: 0.5397 - accuracy: 0.8708


[0.5397418141365051, 0.870757520198822]

In [50]:
best_model_bilstm_doc2vec.evaluate(X_test,y_test)

715/715 [==============================] - 18s 25ms/step - loss: 0.5673 - accuracy: 0.8723


[0.567282497882843, 0.8723320364952087]

In [14]:
best_model_gru_doc2vec.evaluate(X_test,y_test)

  7/715 [..............................] - ETA: 16s - loss: 0.5482 - accuracy: 0.8036

715/715 [==============================] - 12s 17ms/step - loss: 0.4222 - accuracy: 0.8593


[0.422209769487381, 0.8592547178268433]

### Save model

In [ ]:
# Word2Vec
best_model_lstm_word2vec.save(MODEL_FOLDER + sep + 'lstm_word2vec.h5')

In [ ]:
best_model_bilstm_word2vec.save(MODEL_FOLDER + sep + 'bilstm_word2vec.h5')

In [ ]:
best_model_gru_word2vec.save(MODEL_FOLDER + sep + 'gru_word2vec.h5')

In [ ]:
best_model_lstm_doc2vec.save(MODEL_FOLDER + sep + "lstm_doc2vec.h5")

In [51]:
best_model_bilstm_doc2vec.save(MODEL_FOLDER + sep + "bilstm_doc2vec.h5")

In [15]:
best_model_gru_doc2vec.save(MODEL_FOLDER + sep + 'gru_doc2vec.h5')

## Visualize

In [ ]:
plot_optimization_history(study_lstm_word2vec)

In [ ]:
plot_optimization_history(study_bilstm_word2vec)


In [ ]:
plot_optimization_history(study_gru_word2vec)

In [ ]:
html_file_path = "./images/study_lstm_doc2vec_optimize_history.html"
# Plot and save the optimization history plot as an HTML file
ov.plot_optimization_history(study_lstm_doc2vec).write_html(html_file_path)
plot_optimization_history(study_lstm_doc2vec)

In [53]:
html_file_path = "./images/study_bilstm_doc2vec_optimize_history.html"
# Plot and save the optimization history plot as an HTML file
ov.plot_optimization_history(study_bilstm_doc2vec).write_html(html_file_path)
plot_optimization_history(study_bilstm_doc2vec)

In [60]:
html_file_path = "./images/study_gru_doc2vec_optimize_history.html"
# Plot and save the optimization history plot as an HTML file
ov.plot_optimization_history(study_gru_doc2vec).write_html(html_file_path)
plot_optimization_history(study_gru_doc2vec)

## Predict

In [8]:
# Chỉ số từ Index
index = pd.Index(y_train.columns)

# Chuyển đổi Index thành danh sách
labels_list = index.tolist()

# Tạo từ điển ánh xạ
label_mapping = dict(enumerate(labels_list))

# In từ điển ánh xạ
print(label_mapping)

{0: 'Chinh tri Xa hoi', 1: 'Cong nghe', 2: 'Doi song', 3: 'Giai tri', 4: 'Giao duc', 5: 'Khoa hoc', 6: 'Kinh doanh', 7: 'Phap luat', 8: 'Suc khoe', 9: 'The gioi', 10: 'The thao', 11: 'Van hoa'}


In [40]:
# Nhập nội dung văn bản
input_text = """
Uống nước nhớ nguồn, kính trọng những người đi trước vốn là những đạo lý tốt đẹp của dân tộc ta, là yếu tố quan trọng để tạo lập những giá trị bền vững cho tương lai. Từ những ý nghĩa sâu xa đó, sáng ngày 21/10/2023, Trường Dự bị Đại học TP. Hồ Chí Minh đã tổ chức họp mặt, chia tay đối với ông Nguyễn Minh Châu – nhân viên Phòng Công tác Học sinh – Sinh viên về nghỉ hưu theo chế độ. Tham dự buổi họp mặt có sự tham gia của Ban Giám hiệu; Trưởng,…
"""
processed_text = preprocess_text(input_text)

In [41]:
print(processed_text)

['uống_nước_nhớ_nguồn', 'kính_trọng', 'đi', 'vốn', 'đạo_lý', 'tốt_đẹp', 'dân_tộc', 'yếu_tố', 'tạo_lập', 'bền_vững', 'tương_lai', 'ý_nghĩa', 'sâu_xa', 'trường', 'dự_bị_đại_học', 'tổ_chức', 'họp_mặt', 'chia_tay', 'nhân_viên', 'phòng', 'công_tác', 'học_sinh', 'sinh_viên', 'nghỉ', 'hưu', 'chế_độ', 'tham_dự', 'họp_mặt', 'tham_gia', 'ban_giám_hiệu', 'trưởng']


In [42]:
text_token = tokenizer.texts_to_sequences([processed_text])

In [43]:
print(text_token)

[[23745, 5501, 1, 144, 8727, 1517, 734, 430, 6909, 2122, 417, 770, 7330, 2, 6225, 10, 7856, 1019, 223, 32, 141, 14, 90, 389, 1997, 577, 455, 7856, 36, 3246, 181]]


In [44]:
new_padded = pad_sequences(text_token, padding='post', truncating='post')

In [35]:
model = load_model(MODEL_FOLDER + sep + "bilstm_doc2vec.h5")

In [45]:
# Thực hiện dự đoán
predictions = model.predict(new_padded)

# Tạo DataFrame để lưu trữ kết quả
result_df = pd.DataFrame(columns=['Nhãn', 'Phần trăm Độ tin cậy'])

# In ra độ tin cậy của tất cả các nhãn và lưu vào DataFrame
for i, prediction in enumerate(predictions):
    
    # In ra tên nhãn và độ tin cậy tương ứng
    for index, label_name in label_mapping.items():
        
        confidence_percent = prediction[index] * 100
        result_df = result_df.append({'Nhãn': label_name, 'Phần trăm Độ tin cậy': confidence_percent}, ignore_index=True)
# Lấy chỉ số của dòng có giá trị lớn nhất trong cột 'Phần trăm Độ tin cậy'
max_confidence_index = result_df['Phần trăm Độ tin cậy'].idxmax()

# Lấy thông tin của dòng có độ tin cậy cao nhất
max_confidence_row = result_df.loc[max_confidence_index]

# In ra nhãn có độ tin cậy cao nhất và độ tin cậy tương ứng
max_confidence_label = max_confidence_row['Nhãn']
# Định dạng cột 'Phần trăm Độ tin cậy' để thêm đuôi %
result_df['Phần trăm Độ tin cậy'] = result_df['Phần trăm Độ tin cậy'].apply(lambda x: f'{x:.2f}%')

# In ra DataFrame
print(result_df)
print(max_confidence_label)

                Nhãn Phần trăm Độ tin cậy
0   Chinh tri Xa hoi               92.47%
1          Cong nghe                0.01%
2           Doi song                0.02%
3           Giai tri                0.00%
4           Giao duc                5.54%
5           Khoa hoc                0.00%
6         Kinh doanh                1.54%
7          Phap luat                0.17%
8           Suc khoe                0.00%
9           The gioi                0.06%
10          The thao                0.19%
11           Van hoa                0.01%
Chinh tri Xa hoi


## Evaluate model

In [76]:
model = load_model(MODEL_FOLDER + sep + "lstm_doc2vec.h5")

In [77]:
y_pred=model.predict_classes(X_test)
y_pred_onehot = pd.get_dummies(y_pred)

In [78]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_pred_onehot = pd.get_dummies(y_pred)
# Accuracy
accuracy = accuracy_score(y_test, y_pred_onehot)
print(f'Accuracy: {accuracy:.4f}')

# Precision, Recall, and F1-score (calculate for each class and then average)
precision = precision_score(y_test, y_pred_onehot, average='weighted')
recall = recall_score(y_test, y_pred_onehot, average='weighted')
f1 = f1_score(y_test, y_pred_onehot, average='weighted')

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')

Accuracy: 0.8708
Precision: 0.8712
Recall: 0.8708
F1-score: 0.8706


In [79]:
def evaluate_per_label(y_true, y_pred):
    """
    Evaluate precision, recall, and f1-score for each label.

    Parameters:
    - y_true: DataFrame, true labels
    - y_pred: Series or array, predicted labels (integer labels)

    Returns:
    - evaluation_df: DataFrame, containing precision, recall, and f1-score for each label
    """
    # One-hot encode predicted labels
    y_pred_onehot = pd.get_dummies(y_pred)

    # Calculate precision, recall, and f1-score for each label
    precision_per_label = precision_score(y_true, y_pred_onehot, average=None)
    recall_per_label = recall_score(y_true, y_pred_onehot, average=None)
    f1_per_label = f1_score(y_true, y_pred_onehot, average=None)

    # Create a DataFrame to store the results
    evaluation_df = pd.DataFrame({
        'Label': y_true.columns,
        'Precision': precision_per_label,
        'Recall': recall_per_label,
        'F1-score': f1_per_label
    })

    return evaluation_df

# Example usage:
# Assuming y_test is the true labels DataFrame and y_pred is the predicted labels Series or array
evaluation_results = evaluate_per_label(y_test, y_pred)

# Display the evaluation results
print(evaluation_results)

               Label  Precision    Recall  F1-score
0   Chinh tri Xa hoi   0.774953  0.764870  0.769878
1          Cong nghe   0.903834  0.854922  0.878698
2           Doi song   0.814359  0.779195  0.796389
3           Giai tri   0.879452  0.879452  0.879452
4           Giao duc   0.884535  0.921611  0.902693
5           Khoa hoc   0.846154  0.870342  0.858078
6         Kinh doanh   0.850676  0.898846  0.874098
7          Phap luat   0.928174  0.869135  0.897684
8           Suc khoe   0.842046  0.884298  0.862655
9           The gioi   0.921038  0.870005  0.894795
10          The thao   0.962715  0.976722  0.969668
11           Van hoa   0.856721  0.894142  0.875031
